# Задача 3: Рекомендации, какой фрукт купить

# I. Импорт библиотек

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import surprise
import apyori
from apyori import apriori
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Загружаем приготовленный датасет
df_village = pd.read_csv('village_table.csv')

# II. Преобразования

In [3]:
data = df_village.copy()

In [4]:
data.head()

,village,year,family,category,fruit,grade,order_date,delivery_date,ordered,shipped,leadtime
0,Robinhood_village,2020,family_244,category_25,fruit_37,165,2019-12-30,2020-01-03,1650.0,1663.20,4 days
1,Robinhood_village,2020,family_127,category_14,fruit_62,250,2019-12-30,2020-01-10,1000.0,1060.00,11 days
2,Robinhood_village,2020,family_133,category_16,fruit_48,330,2019-12-30,2020-01-08,1320.0,1336.50,9 days
3,Robinhood_village,2020,family_133,category_25,fruit_49,250,2019-12-30,2020-01-08,2500.0,2508.75,9 days
4,Robinhood_village,2020,family_133,category_4,fruit_52,315,2019-12-30,2020-01-08,4725.0,4759.65,9 days


In [5]:
data.order_date.max()

'2022-02-20'

Разделим выборку хронологически на обучающую и валидационную

In [6]:
train = data[data.order_date < '2022-02-12']
val = data[data.order_date >= '2022-02-12']

In [7]:
train = train[['family','fruit','ordered']]
val = val[['family','fruit','ordered']]

In [8]:
# Данные по заказам, которые нужно суммировать
train = train.groupby(['family','fruit']).agg(['count'])
train.reset_index(inplace=True)
train.columns = train.columns.droplevel(1)
train.columns = ['family','fruit','order_count']

val = val.groupby(['family','fruit']).agg(['count'])
val.reset_index(inplace=True)
val.columns = val.columns.droplevel(1)
val.columns = ['family','fruit','order_count']

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18346 entries, 0 to 18345
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   family       18346 non-null  object
 1   fruit        18346 non-null  object
 2   order_count  18346 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 430.1+ KB


In [10]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   family       644 non-null    object
 1   fruit        644 non-null    object
 2   order_count  644 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 15.2+ KB


In [11]:
train['order_count'].describe()

count    18346.000000
mean        13.240434
std         39.971358
min          1.000000
25%          1.000000
50%          4.000000
75%         11.000000
max       1531.000000
Name: order_count, dtype: float64

Создаем перечень часто покупаемых фруктов семьями в обучающей выборке  
В виде строки разделенных пробелом фруктов, а также в виде таблицы по семьям в виде вложенного в ячейку списка

In [12]:
train_popular = train[train.order_count > 10].groupby('family')['fruit']
train_pop_str = train_popular.apply(lambda r: ' '.join([str(A) for A in r]))
train_family_list = list(train_pop_str.index)
train_pop_list = train[train.order_count > 11].groupby('family')['fruit'].apply(list)

In [13]:
train_pop_list

family
family_0      [fruit_4470, fruit_4471, fruit_4472, fruit_447...
family_1      [fruit_133, fruit_136, fruit_138, fruit_180, f...
family_10                                 [fruit_57, fruit_883]
family_100    [fruit_1020, fruit_1286, fruit_136, fruit_145,...
family_101    [fruit_1149, fruit_182, fruit_189, fruit_345, ...
                                    ...                        
family_95     [fruit_1168, fruit_1283, fruit_136, fruit_154,...
family_96     [fruit_1113, fruit_124, fruit_132, fruit_1424,...
family_97     [fruit_1068, fruit_1135, fruit_1149, fruit_115...
family_98                               [fruit_1478, fruit_917]
family_99     [fruit_1118, fruit_1119, fruit_1222, fruit_124...
Name: fruit, Length: 273, dtype: object

In [14]:
# Создаем перечень купленных фруктов семьями в валидационной выборке
val_grouped = val.groupby('family')['fruit']
val_grouped = val_grouped.apply(lambda r: ' '.join([str(A) for A in r]))

# Выведем список купленных фруктов в обучающей выборке
val_family_list = list(val_grouped.index)

# Таблица по семьям в виде вложенного в ячейку списка
val_list = val.groupby('family')['fruit'].apply(list)

In [15]:
val_list

family
family_0      [fruit_1517, fruit_2853, fruit_4478, fruit_448...
family_100                 [fruit_1202, fruit_3117, fruit_4169]
family_101                                         [fruit_1149]
family_102                                         [fruit_3568]
family_105                 [fruit_1318, fruit_2013, fruit_2599]
                                    ...                        
family_92                                          [fruit_3455]
family_93                                [fruit_189, fruit_621]
family_96     [fruit_1113, fruit_143, fruit_1567, fruit_171,...
family_97     [fruit_1774, fruit_1862, fruit_2209, fruit_3, ...
family_99     [fruit_1118, fruit_1222, fruit_155, fruit_2472...
Name: fruit, Length: 128, dtype: object

In [16]:
val_grouped

family
family_0      fruit_1517 fruit_2853 fruit_4478 fruit_4481 fr...
family_100                     fruit_1202 fruit_3117 fruit_4169
family_101                                           fruit_1149
family_102                                           fruit_3568
family_105                     fruit_1318 fruit_2013 fruit_2599
                                    ...                        
family_92                                            fruit_3455
family_93                                   fruit_189 fruit_621
family_96     fruit_1113 fruit_143 fruit_1567 fruit_171 frui...
family_97     fruit_1774 fruit_1862 fruit_2209 fruit_3 fruit...
family_99     fruit_1118 fruit_1222 fruit_155 fruit_2472 fru...
Name: fruit, Length: 128, dtype: object

In [17]:
# Перечень семей в обучающей выборке
len(train_family_list)

277

In [18]:
# Перечень семей в валидационной выборке
len(val_family_list)

128

In [19]:
# Семьи, которые не покупали в период, выбранный для обучающей выборки
list(set(val_family_list) - set(train_family_list))

['family_236',
 'family_399',
 'family_292',
 'family_395',
 'family_322',
 'family_379',
 'family_161']

In [20]:
val[val.family == 'family_399'].family.iloc[0] not in train_family_list
# not in train_family_list

True

In [21]:
# Выводим строку часто покупаемых фруктов для семьи 100
train_pop_str['family_100']

'fruit_1020 fruit_1286 fruit_136 fruit_145 fruit_176 fruit_1776 fruit_19 fruit_2085 fruit_252 fruit_283 fruit_29 fruit_2926 fruit_2991 fruit_331 fruit_3329 fruit_3518 fruit_56 fruit_57'

### Ранкинг

Выведем ранкинг наиболее часто покупаемых фруктов в деревне:

In [22]:
train_total_rank = train.copy()
train_total_rank = train_total_rank.groupby(['fruit']).sum()
train_total_rank = train_total_rank.sort_values(['order_count'], ascending=False)

In [23]:
train_total_rank.head(3)

,order_count
fruit,
fruit_19,7912
fruit_59,7585
fruit_189,6359


In [24]:
train_total_top_3 = list(train_total_rank.index)[:3]

In [25]:
train_total_top_5 = list(train_total_rank.index)[:5]

Для более детального прогнозирования можно вывести наиболее популярных фруктов для каждой семьи в обучающей выборке

Создадим список наиболее популярных фруктов в каждой семье в трех форматах (для более удобного пользования в дальнейших действиях)

In [76]:
train_rank = train.copy()
train_top_3_ranking = pd.DataFrame()
for i in range(0,3):
    test_df = train_rank.copy()
    test_df = test_df.sort_values(['family','order_count'], ascending=False)
    test_df = test_df.groupby(['family']).nth(i)
    train_top_3_ranking = pd.concat([train_top_3_ranking, test_df])

train_top_3_ranking.reset_index(inplace=True)
train_ranking = train_top_3_ranking.groupby('family')['fruit']

train_ranking = train_ranking.apply(lambda r: ' '.join([str(A) for A in r]))
train_top_list = train_top_3_ranking.groupby('family')['fruit'].apply(list)
train_top_df = train_top_list.reset_index()
train_top_df.columns = ['family','top']

В виде серии с разбитеем на семьи и одной ячейкой. Фрукты разделены пробелами

In [77]:
train_ranking.head()

family
family_0      fruit_4478 fruit_4482 fruit_4481
family_1         fruit_663 fruit_182 fruit_133
family_10         fruit_57 fruit_883 fruit_478
family_100       fruit_1776 fruit_176 fruit_19
family_101        fruit_182 fruit_59 fruit_189
Name: fruit, dtype: object

В виде серии с разбитеем на семьи и одной ячейкой в виде списка. Фрукты разделены запятыми

In [78]:
train_top_list.head()

family
family_0      [fruit_4478, fruit_4482, fruit_4481]
family_1         [fruit_663, fruit_182, fruit_133]
family_10         [fruit_57, fruit_883, fruit_478]
family_100       [fruit_1776, fruit_176, fruit_19]
family_101        [fruit_182, fruit_59, fruit_189]
Name: fruit, dtype: object

В виде датафрейма с вложенным списком в ячейке

In [79]:
train_top_df.head()

,family,top
0,family_0,"[fruit_4478, fruit_4482, fruit_4481]"
1,family_1,"[fruit_663, fruit_182, fruit_133]"
2,family_10,"[fruit_57, fruit_883, fruit_478]"
3,family_100,"[fruit_1776, fruit_176, fruit_19]"
4,family_101,"[fruit_182, fruit_59, fruit_189]"


Список купленных фруктов в валидационной выборке будет служить для определения точности предсказания

In [37]:
val_grouped.head()

family
family_0      fruit_1517 fruit_2853 fruit_4478 fruit_4481 fr...
family_100                     fruit_1202 fruit_3117 fruit_4169
family_101                                           fruit_1149
family_102                                           fruit_3568
family_105                     fruit_1318 fruit_2013 fruit_2599
Name: fruit, dtype: object

In [38]:
val_list.head()

family
family_0      [fruit_1517, fruit_2853, fruit_4478, fruit_448...
family_100                 [fruit_1202, fruit_3117, fruit_4169]
family_101                                         [fruit_1149]
family_102                                         [fruit_3568]
family_105                 [fruit_1318, fruit_2013, fruit_2599]
Name: fruit, dtype: object

In [39]:
val_df = val_list.reset_index()
val_df.columns = ['family','true_top']

In [40]:
val_df.head()

,family,true_top
0,family_0,"[fruit_1517, fruit_2853, fruit_4478, fruit_448..."
1,family_100,"[fruit_1202, fruit_3117, fruit_4169]"
2,family_101,[fruit_1149]
3,family_102,[fruit_3568]
4,family_105,"[fruit_1318, fruit_2013, fruit_2599]"


# III. Создание рекомендации 

### 1. Построение ассоциативных правил

Правила можно изменять, модифицируя support, confidence или lift.

In [102]:
# Создаем ассоциативные правила
association_rules = apyori.apriori(train_pop_str.apply(lambda r: r.split(' ')), 
                                   min_support=0.025, 
                                   min_confidence=0.1, 
                                   min_lift=3, 
                                   min_length=2)

Ассоциативые правила необходимо преобразовать в таблицу рекомендаций, чтобы правила были более удобными для прочиеня

In [103]:
# Функция для вывода рекомендации на экран
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

После экспериментов были выбраны параметры, которые улучшали precision  
Посмотрим на количество созданных правил

In [104]:
len(asr_df)

10687

In [54]:
asr_df.sort_values(['lift','confidence'], ascending=False).head(10)

,from,to,confidence,support,lift
1486,fruit_469,fruit_642 fruit_81,1.0,0.028881,34.625
1488,fruit_469,fruit_642 fruit_83,1.0,0.028881,34.625
3026,fruit_469,fruit_501 fruit_642 fruit_81,1.0,0.028881,34.625
3028,fruit_469,fruit_501 fruit_642 fruit_83,1.0,0.028881,34.625
3046,fruit_469,fruit_642 fruit_81 fruit_76,1.0,0.028881,34.625
3048,fruit_469,fruit_642 fruit_83 fruit_76,1.0,0.028881,34.625
3050,fruit_469,fruit_642 fruit_81 fruit_77,1.0,0.028881,34.625
3052,fruit_469,fruit_642 fruit_83 fruit_77,1.0,0.028881,34.625
3053,fruit_469,fruit_642 fruit_81 fruit_79,1.0,0.028881,34.625
3055,fruit_469,fruit_642 fruit_83 fruit_79,1.0,0.028881,34.625


In [99]:
asr_df.sort_values(['to'], ascending=False).head(10)

,from,to,confidence,support,lift
3055,fruit_469,fruit_642 fruit_83 fruit_79,1.0,0.028881,34.625
3052,fruit_469,fruit_642 fruit_83 fruit_77,1.0,0.028881,34.625
3048,fruit_469,fruit_642 fruit_83 fruit_76,1.0,0.028881,34.625
1488,fruit_469,fruit_642 fruit_83,1.0,0.028881,34.625
3053,fruit_469,fruit_642 fruit_81 fruit_79,1.0,0.028881,34.625
3050,fruit_469,fruit_642 fruit_81 fruit_77,1.0,0.028881,34.625
3046,fruit_469,fruit_642 fruit_81 fruit_76,1.0,0.028881,34.625
1486,fruit_469,fruit_642 fruit_81,1.0,0.028881,34.625
3028,fruit_469,fruit_501 fruit_642 fruit_83,1.0,0.028881,34.625
3026,fruit_469,fruit_501 fruit_642 fruit_81,1.0,0.028881,34.625


In [91]:
asr_df = asr_df.sort_values(['lift','confidence'], ascending=False).head(10)

In [98]:
asr_df.iloc[1]

from                   fruit_469
to            fruit_642 fruit_83
confidence                   1.0
support                 0.028881
lift                      34.625
Name: 1488, dtype: object

Для какого количества фруктов созданы рекомендации:

In [105]:
len(asr_df['from'].value_counts())

78

Всего в валидационной выборке уникальных фруктов

In [106]:
val.fruit.nunique()

494

### 2. Предсказания

Создаем датафрейм с предсказаниями на основе рекоммендации априори. 
* Выбираем правила на основании фруктов, которые семья относительно часто покупала до этого (train_pop_str). 
* В случае, если в ассоцитивых правилах не было фруктов, которые покупала семья, предлагаем семье фрукты, которые ранее семья чаще всего покупала. 
* Есть также семьи, которые не покупали в период, который был выбран за обучающую выборку. Для таких семей можно порекомендовать наиболее популярные продукты во всей деревне (train_total_top_3)

In [80]:
train_top_df.head()

,family,top
0,family_0,"[fruit_4478, fruit_4482, fruit_4481]"
1,family_1,"[fruit_663, fruit_182, fruit_133]"
2,family_10,"[fruit_57, fruit_883, fruit_478]"
3,family_100,"[fruit_1776, fruit_176, fruit_19]"
4,family_101,"[fruit_182, fruit_59, fruit_189]"


In [81]:
# Это три из наиболее популярных фруктов, которые можно рекомендовать новым семьям
train_total_top_3

['fruit_19', 'fruit_59', 'fruit_189']

In [82]:
# Семьи, которые не покупали в периоде обучающей выбоки. Т.е. новые клиенты
missing = list(set(val_family_list) - set(train_family_list))

In [83]:
missing

['family_236',
 'family_399',
 'family_292',
 'family_395',
 'family_322',
 'family_379',
 'family_161']

#### 2.1. Рекомендации на основании выбранного фрукта

Принцип действия такой: если семье нравится один фрукт, то предложим фрукты, которые наиболее часто ассоциируются с этим фруктом.  
Пример рекомендации для кого-то, кто любит (в нашем случае "покупает") выбранный продукт

In [72]:
asr_df[asr_df['from'] == 'fruit_407']

,from,to,confidence,support,lift
377,fruit_407,fruit_471,0.466667,0.025271,7.181481
378,fruit_407,fruit_57,0.6,0.032491,4.616667


Создадим датафрейм с рекомендациями по выбранному фрукту

In [ ]:
test = ['fruit_407']

In [69]:
recom = pd.DataFrame(columns = ['fruit','recommended'])
list_from = []

for fruit in asr_df['from'].unique():
# for fruit in test:
    element = []
    base = asr_df[asr_df['from'] == fruit]
    title_1 = fruit
    x = 0
    for i in base['to'].unique():
        if i not in element:
            element.append(i)
            x += 1
            if x > 5:
                break
        else:
            continue
    new_row = {'fruit':title_1, 'recommended':element}

    recom = recom.append(new_row,ignore_index=True)

In [71]:
# Так выглядит таблица рекомендаций
recom.head()

,fruit,recommended
0,fruit_123,"[fruit_133, fruit_154, fruit_19, fruit_193, fr..."
1,fruit_124,"[fruit_142, fruit_177, fruit_193, fruit_200, f..."
2,fruit_1272,"[fruit_133, fruit_154, fruit_2150, fruit_271, ..."
3,fruit_131,"[fruit_154, fruit_2366, fruit_2366 fruit_59]"
4,fruit_132,"[fruit_189, fruit_193, fruit_568, fruit_133 fr..."


In [ ]:
recom.to_csv('recomm_fruit.csv', index=False)

#### 2.2. Рекомендации на основании прошлых заказов семьи

Создание датафрейма предсказаний

In [92]:
missing = list(set(val_family_list) - set(train_family_list))

result = pd.DataFrame(columns = ['family','recommended'])
for family in val_family_list:

    element = []
    x = 0
    y = 0
    list_1 = train_top_list.loc[family]
    list_2 = asr_df['from'].unique()
    common = list(set(list_1).intersection(list_2))
    
    if family in missing:
        for l in train_total_top_3:
            element.append(l)
        new_row = {'family':family, 'recommended':element}

    elif len(common) > 0:
        for i in asr_df.index:
            if asr_df['from'].loc[i].split(' ')[0] in train_pop_str.loc[family]:
                if asr_df['to'].loc[i] not in element:
                    element.append(asr_df['to'].loc[i])
                    x += 1
                    if x >= 3:
                        break
                else:
                    continue
        new_row = {'family':family, 'recommended':element}
    else:
        for j in train_top_df[train_top_df.family == family].top:
            for k in j:
                element.append(k)
        new_row = {'family':family, 'recommended':element}       
        
    result = result.append(new_row,ignore_index=True)

In [107]:
result.head()

,family,recommended
0,family_0,"[fruit_4478, fruit_4482, fruit_4481]"
1,family_100,"[fruit_1776, fruit_176, fruit_19]"
2,family_101,"[fruit_182, fruit_59, fruit_189]"
3,family_102,"[fruit_38, fruit_132, fruit_200]"
4,family_105,"[fruit_834, fruit_19, fruit_138]"


In [89]:
# Проверяем результат
result[result.family == 'family_101']

,family,recommended
2,family_101,"[fruit_144, fruit_174, fruit_175]"


In [ ]:
result.to_csv('recommendation.csv', index=False)

### 3. Оценка качества

In [94]:
evaluation = pd.DataFrame(columns = ['family','fruit','eval'])

true_positive = 0
false_positive = 0
for fam in val_family_list:
    for i in result[result.family == fam]['recommended'].iloc[0]:
        if i in val_df[val_df.family == fam].true_top.iloc[0]:
            true_positive += 1
        else:
            false_positive += 1
            
precision = true_positive/(true_positive+false_positive)
print('true positive = ', true_positive)
print('false positive = ', false_positive)
print('precision = ', precision)


true positive =  49
false positive =  335
precision =  0.12760416666666666


In [ ]:
result.to_csv('recommendation.csv', index=False)

# III Итоги по задаче 3

Главной целью задачи 3 было построение рекомендаций для запросов на основании исторических данных, а также на основании запрашиваемого фрукта
* Был использован алгоритм априори
* Были подобраны параметры support, confidence, lift для улучшения метрики precision
* В данных нет информации о том, нравится ли семье покупаемый фрукт, но можно использовать количество закупок
* Минусом использования данного метода решения поставленной бизнес задачи то, что семей, у которых ассортимент очень большой, а также имеется различная периодичность закупок различных фруктов, рекоммендация может не сработать в ближайший период, а алгоритм выдает ограниченнон количество рекомендаций.